In [52]:
# ================= Step 1: Connect to GPU ================= #
!/opt/bin/nvidia-smi


Mon Jan  3 17:33:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
# ================= Step 2.1: Connect to Google Drive ================= #
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.27-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.27-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.27-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

# 新段落

In [2]:
# ================= Step 2.2: Connect to Google Drive ================= #
!mkdir -p drive
!google-drive-ocamlfuse drive

In [10]:
# ================= Step 3.1: Transform data to colab ================= #
%mkdir /content/datasets/
%cp -r -vv /content/drive/datasets/ipalm/formed_data/depth2edge_synth.zip /content/datasets/
%cp -r -vv /content/drive/datasets/ipalm/formed_data/depth2edge_real.zip /content/datasets/
%cp -r -vv /content/drive/datasets/ipalm/formed_data/depth2final_real_all_depth.zip /content/datasets/

mkdir: cannot create directory ‘/content/datasets/’: File exists
'/content/drive/datasets/ipalm/formed_data/depth2edge_synth.zip' -> '/content/datasets/depth2edge_synth.zip'
'/content/drive/datasets/ipalm/formed_data/depth2edge_real.zip' -> '/content/datasets/depth2edge_real.zip'
'/content/drive/datasets/ipalm/formed_data/depth2final_real_all_depth.zip' -> '/content/datasets/depth2final_real_all_depth.zip'


In [ ]:
# ================= Step 3.2: Unzip Data ================= #
%cd /content/datasets/
%ls
!unzip depth2edge_synth
!unzip depth2edge_real
!unzip depth2final_real_all_depth

In [12]:
# ================= Step 4: Configuration ================= #
%cd /content/drive/IPALM-code-ICRA/Line3/synth_data/
!pip install -r requirements.txt

/content/drive/IPALM-code-ICRA/Line3/synth_data


Train on Synthetic

In [59]:
# ================= Step 5.1: Start Train Segmentation ================= #
%cd /content/drive/02456-final-project/Final_project/Semantic_Segmentation/synth_data/
!bash ./scripts/train_pix2pix.sh

/content/drive/02456-final-project/Final_project/Semantic_Segmentation/synth_data
+ python train.py --dataroot /content/datasets/depth2edge_synth/ --dataroot_target /content/datasets/depth2edge_synth/ --name depth2edge_synth --model our --netG encoder_decoder_256 --netD LeNet --direction AtoB --lambda_entropy 100 --dataset_mode onehotAligned --norm batch --pool_size 0 --input_nc 1 --output_nc 3 --batch_size 8 --checkpoints_dir /content/ --save_epoch_freq 1 --gpu_ids 0 --n_epochs 100 --n_epochs_decay 100
----------------- Options ---------------
               batch_size: 8                             	[default: 1]
                    beta1: 0.5                           
          checkpoints_dir: /content/                     	[default: ./checkpoints]
           continue_train: False                         
                crop_size: 256                           
                 dataroot: /content/datasets/depth2edge_synth/	[default: None]
          dataroot_target: /content/datase

Train on Synthetic, test on Synthetic

In [64]:
# ================= Step 5.2: Start Test Segmentation ================= #
%cd /content/drive/02456-final-project/Final_project/Test/synth_data/
!python test_phase1.py --dataroot /content/datasets/depth2edge_synth/ --name depth2edge_synth --model our --netG encoder_decoder_256 --netD LeNet --direction AtoB --dataset_mode onehotAligned --norm batch --checkpoints_dir /content/ --results_dir /content/result/ --input_nc 1 --output_nc 3 --epoch 199

/content/drive/02456-final-project/Final_project/Test/synth_data
----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: /content/                     	[default: ./checkpoints]
                crop_size: 256                           
                 dataroot: /content/datasets/depth2edge_synth/	[default: None]
          dataroot_target: None                          
             dataset_mode: onehotAligned                 	[default: aligned]
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: 199                           	[default: latest]
                     eval: False                         
                  gpu_ids: -1                            
                init_gain: 0.02                          
                init_type: normal                       

Trian on Synthetic; Test on Real

In [76]:
%cd /content/drive/02456-final-project/Final_project/Test/synth_data/
!python test_phase1.py --dataroot /content/datasets/depth2edge_real/ --name depth2edge_synth --model our --netG encoder_decoder_256 --netD LeNet --direction AtoB --dataset_mode onehotAligned --norm batch --checkpoints_dir /content/ --results_dir /content/result/ --input_nc 1 --output_nc 3 --epoch 199

/content/drive/02456-final-project/Final_project/Test/synth_data
----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: /content/                     	[default: ./checkpoints]
                crop_size: 256                           
                 dataroot: /content/datasets/depth2edge_real/	[default: None]
          dataroot_target: None                          
             dataset_mode: onehotAligned                 	[default: aligned]
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: 199                           	[default: latest]
                     eval: False                         
                  gpu_ids: -1                            
                init_gain: 0.02                          
                init_type: normal                        

DA

In [66]:
# ================= Step 5.3: Start Train DA ================= #
%cd /content/drive/02456-final-project/Final_project/Domain_Adaptation/synth_data/
!bash python train.py --dataroot /content/datasets/depth2edge_synth/ --dataroot_target /content/datasets/depth2final_real_all_depth/ --name DA --model our --netG encoder_decoder_256 --netD LeNet --direction AtoB --lambda_entropy 20 --lambda_DA 100 --dataset_mode onehotAligned --norm batch --pool_size 0 --input_nc 1 --output_nc 3 --batch_size 8 --checkpoints_dir /content/ --save_epoch_freq 1 --gpu_ids 0 --n_epochs 30 --n_epochs_decay 30 --continue_train --epoch 199 

/content/drive/02456-final-project/Final_project/Domain_Adaptation/synth_data
----------------- Options ---------------
               batch_size: 8                             	[default: 1]
                    beta1: 0.5                           
          checkpoints_dir: /content/                     	[default: ./checkpoints]
           continue_train: True                          	[default: False]
                crop_size: 256                           
                 dataroot: /content/datasets/depth2edge_synth/	[default: None]
          dataroot_target: /content/datasets/depth2final_real_all_depth/	[default: None]
             dataset_mode: onehotAligned                 	[default: aligned]
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: 1                             
            display_ncols: 4                             


Test DA on Real data

In [77]:
# ================= Step 5.4: Start Test DA ================= #
%cd /content/drive/02456-final-project/Final_project/Test/synth_data/
!python test_phase1.py --epoch 50 --dataroot /content/datasets/depth2edge_real/ --name DA --model our --netG encoder_decoder_256 --netD LeNet --direction AtoB --dataset_mode onehotAligned --norm batch --checkpoints_dir /content/ --results_dir /content/result/ --input_nc 1 --output_nc 3 

/content/drive/02456-final-project/Final_project/Test/synth_data
----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: /content/                     	[default: ./checkpoints]
                crop_size: 256                           
                 dataroot: /content/datasets/depth2edge_real/	[default: None]
          dataroot_target: None                          
             dataset_mode: onehotAligned                 	[default: aligned]
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: 50                            	[default: latest]
                     eval: False                         
                  gpu_ids: -1                            
                init_gain: 0.02                          
                init_type: normal                        